#6.6 RetrievalQA

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA


In [9]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

@traceable
def generate_response(prompt):
    return llm.invoke(prompt)


In [10]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
)

generate_response("Describe Victory Mansions")


AIMessage(content="Victory Mansions is a fictional location, most notably featured in Agatha Christie's 1940 novel *Ten Little Indians* (also known as *And Then There Were None*).  It's a large, isolated mansion on a remote island off the coast of Devon, England.  \n\nThe description emphasizes its imposing and somewhat sinister nature:\n\n* **Isolated Location:**  Its remoteness contributes significantly to the claustrophobic and suspenseful atmosphere of the novel.  The inhabitants are completely cut off from the outside world.\n\n* **Grand but Decaying:** While initially described as grand and impressive, a sense of neglect and decay permeates the description. This adds to the feeling of unease and impending doom.\n\n* **Mysterious History:** Hints of a dark past are suggested, adding to the overall mystery.  The exact nature of this history isn't explicitly detailed, enhancing the suspense.\n\n* **Symbolic Significance:**  The mansion itself becomes a symbolic representation of the